In [ ]:
import pygame
import sys
import random

# Initialize Pygame
pygame.init()

# Initial velocity flag
init_vel = True

# Screen dimensions
WIDTH, HEIGHT = 800, 600
BALL_SIZE = 20
PADDLE_WIDTH, PADDLE_HEIGHT = 10, 100
FPS = 60

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

# Create screen
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Pong - Local Multiplayer")
clock = pygame.time.Clock()

# Paddle positions
left_paddle = pygame.Rect(50, HEIGHT // 2 - PADDLE_HEIGHT // 2, PADDLE_WIDTH, PADDLE_HEIGHT)
right_paddle = pygame.Rect(WIDTH - 60, HEIGHT // 2 - PADDLE_HEIGHT // 2, PADDLE_WIDTH, PADDLE_HEIGHT)

# Ball position
ball = pygame.Rect(WIDTH // 2 - BALL_SIZE // 2, HEIGHT // 2 - BALL_SIZE // 2, BALL_SIZE, BALL_SIZE)

# Ball velocity
def reset_ball():
    ball.center = (WIDTH // 2, HEIGHT // 2 + HEIGHT * random.randint(-2, 2) // 8)
    global init_vel
    init_vel = True
    return random.choice([-4, 4]), random.choice([-2, 2])

ball_vel_x, ball_vel_y = reset_ball()

# Score variables
left_score = 0
right_score = 0
font = pygame.font.Font(None, 74)

# Movement functions
def move_ball():
    global ball_vel_x, ball_vel_y, left_score, right_score
    ball.x += ball_vel_x
    ball.y += ball_vel_y

    # Wall collisions
    if ball.top <= 0 or ball.bottom >= HEIGHT:
        ball_vel_y *= -1

    # Paddle collisions
    resolve_collision(left_paddle)
    resolve_collision(right_paddle)

    # Scoring
    if ball.left <= 0:
        right_score += 1
        reset_ball_with_pause()
    elif ball.right >= WIDTH:
        left_score += 1
        reset_ball_with_pause()

def resolve_collision(paddle):
    global ball_vel_x, ball_vel_y, init_vel

    if ball.colliderect(paddle):
        # Determine collision side
        if (ball_vel_x > 0 and ball.right > paddle.left) or (ball_vel_x < 0 and ball.left < paddle.right):
            # Check if collision is vertical (i.e., from left/right)
            ball_vel_x *= -1

            # Adjust vertical bounce based on where it hit the paddle
            relative_intersect_y = (paddle.centery - ball.centery)
            normalized = relative_intersect_y / (PADDLE_HEIGHT / 2)
            bounce_angle = normalized * -80  # Max angle in degrees
                                             # Using negative value for ball to bounce outward

            # Use vector rotation to get Y velocity
            new_velocity = pygame.math.Vector2(7, 0).rotate(bounce_angle)
            ball_vel_y = int(new_velocity.y)

            if init_vel:
                ball_vel_x *= 2
                ball_vel_y *= 2
                init_vel = False

            # Prevent ball from staying stuck inside paddle
            if ball_vel_x > 0:
                ball.left = paddle.right + 1
            else:
                ball.right = paddle.left - 1

def reset_ball_with_pause():
    global ball_vel_x, ball_vel_y
    pygame.time.delay(700)
    ball_vel_x, ball_vel_y = reset_ball()

def move_paddles():
    keys = pygame.key.get_pressed()

    # Left paddle (Player 1: W/S)
    if keys[pygame.K_w] and left_paddle.top > 0:
        left_paddle.y -= 10
    if keys[pygame.K_s] and left_paddle.bottom < HEIGHT:
        left_paddle.y += 10

    # Right paddle (Player 2: UP/DOWN)
    if keys[pygame.K_UP] and right_paddle.top > 0:
        right_paddle.y -= 10
    if keys[pygame.K_DOWN] and right_paddle.bottom < HEIGHT:
        right_paddle.y += 10

def draw_objects():
    screen.fill(BLACK)
    
    # Draw paddles and ball
    pygame.draw.rect(screen, WHITE, left_paddle)
    pygame.draw.rect(screen, WHITE, right_paddle)
    pygame.draw.ellipse(screen, WHITE, ball)
    
    # Draw center line
    pygame.draw.aaline(screen, WHITE, (WIDTH // 2, 0), (WIDTH // 2, HEIGHT))

    # Display scores
    left_text = font.render(str(left_score), True, WHITE)
    right_text = font.render(str(right_score), True, WHITE)
    screen.blit(left_text, (WIDTH // 2 - 100, 20))
    screen.blit(right_text, (WIDTH // 2 + 60, 20))

# Main loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    move_paddles()
    move_ball()

    draw_objects()

    pygame.display.flip()
    clock.tick(FPS)

pygame.quit()
sys.exit()

SystemExit: 